In [1]:
import sys
import os
from z3 import *
from BitVector import *
import pdb

path = os.getcwd()
rootpath = path[:path.rfind('src') + 4]
sys.path.append(f'{rootpath}/PyWorthy')

In [2]:
benchmark_models = [
    "Benchmarks/Blasted_Real/blasted_case47.cnf",  # 0
    "Benchmarks/Blasted_Real/blasted_case110.cnf",
    "Benchmarks/V7/s820a_7_4.cnf",
    "Benchmarks/V15/s820a_15_7.cnf",
    "Benchmarks/V3/s1238a_3_2.cnf",
    "Benchmarks/V3/s1196a_3_2.cnf",  # 5
    "Benchmarks/V15/s832a_15_7.cnf",
    "Benchmarks/Blasted_Real/blasted_case_1_b12_2.cnf",
    "Benchmarks/Blasted_Real/blasted_squaring16.cnf",
    "Benchmarks/Blasted_Real/blasted_squaring7.cnf",
    "Benchmarks/70.sk_3_40.cnf",  # 10
    "Benchmarks/ProcessBean.sk_8_64.cnf",
    "Benchmarks/56.sk_6_38.cnf",
    "Benchmarks/35.sk_3_52.cnf",
    "Benchmarks/80.sk_2_48.cnf",
    "Benchmarks/7.sk_4_50.cnf",  # 15
    "Benchmarks/doublyLinkedList.sk_8_37.cnf",
    "Benchmarks/19.sk_3_48.cnf",
    "Benchmarks/29.sk_3_45.cnf",
    "Benchmarks/isolateRightmost.sk_7_481.cnf",
    "Benchmarks/17.sk_3_45.cnf",  # 20
    "Benchmarks/81.sk_5_51.cnf",
    "Benchmarks/LoginService2.sk_23_36.cnf",
    "Benchmarks/sort.sk_8_52.cnf",
    "Benchmarks/parity.sk_11_11.cnf",
    "Benchmarks/77.sk_3_44.cnf",  # 25
    "Benchmarks/20.sk_1_51.cnf",
    "Benchmarks/enqueueSeqSK.sk_10_42.cnf",
    "Benchmarks/karatsuba.sk_7_41.cnf",
    # "Benchmarks/diagStencilClean.sk_41_36.cnf",
    "Benchmarks/tutorial3.sk_4_31.cnf"
]

In [3]:
class program_s:
    def __init__(self, name):
        self.name = name
        self.var_num = 0
        self.clauses_num = 0
        self.vars = list()
        self.clauses = list()

In [4]:
def load_cnf(model_file, model_name):
    program = program_s(model_name)
    with open(model_file, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith('c'): continue
            if line.startswith('p'):
                _1, _2, var_num, clause_num = line.split(' ')
                program.var_num, program.clauses_num = int(var_num), int(
                    clause_num)
                for i in range(program.var_num + 1):
                    program.vars.append(Bool(f'v{i}'))
                continue
            vs = list(map(lambda i: int(i), line.split(' ')))[:-1]
            vs_ref = [
                program.vars[abs(v)] if v > 0 else (Not(program.vars[abs(v)]))
                for v in vs
            ]
            program.clauses.append(Or(*vs_ref))
    return program

In [5]:
def load_over_sampling(model_name):
    f_name = f'{rootpath}../memo/{model_name}.cnf.memo'
    inits = list()
    with open(f_name, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            bv = BitVector(bitstring=line)
            inits.append(bv)
    return inits

In [6]:
model_id = 0
model_file = f'{rootpath}../{benchmark_models[model_id]}'
model_name = model_file[model_file.rfind('/') + 1:model_file.rfind('.cnf')]
program = load_cnf(model_file, model_name)

In [7]:
inits = load_over_sampling(model_name)

In [10]:
from sklearn.cluster import KMeans

In [11]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(inits)

In [27]:
import numpy as np
np.where(kmeans.labels_ == 1)[0]

array([16, 23, 62, 63, 64, 70, 71, 86, 89, 97])

In [23]:
kmeans.labels_

array([2, 4, 5, 3, 4, 5, 5, 6, 9, 3, 5, 9, 5, 5, 6, 2, 1, 3, 0, 3, 2, 3,
       3, 1, 0, 3, 9, 6, 6, 5, 0, 4, 4, 0, 4, 6, 0, 4, 2, 4, 4, 6, 0, 0,
       0, 0, 4, 4, 4, 3, 3, 3, 0, 4, 6, 6, 2, 2, 5, 5, 0, 0, 1, 1, 1, 8,
       7, 6, 6, 7, 1, 1, 7, 9, 6, 6, 7, 7, 8, 4, 6, 8, 4, 7, 8, 6, 1, 6,
       6, 1, 7, 0, 8, 8, 9, 7, 3, 1, 3, 7], dtype=int32)

In [28]:
np.random.choice(np.where(kmeans.labels_ == 1)[0])

62